# Run Garak Red-Team Evaluation

This notebook is the third step in the demo flow:

1. **01-red-team-prompt-generation.ipynb** — generates adversarial prompts via an sdg_hub flow → saves JSON to `$XDG_DATA_HOME`
2. **02-garak-to-sdg.ipynb** — converts that JSON into garak's expected format (`trait_typology.json` + `intent_stubs/*.txt`) under `$XDG_DATA_HOME/garak/data/cas/`
3. **03-run-garak.ipynb** *(this notebook)* — runs garak against the target LLM using `data/garak.yaml` and reports the output file paths

> **OpenShift AI (remote cluster):** `data/garak.yaml` points the target, detector, and TAP models to `http://localhost:8080/v1`. If the model has no external route, port-forward it before running this notebook:
> ```bash
> oc port-forward -n stuart-testing pod/ilyagusevgemma-2-9b-it-abliterated-predictor-54597b8fb-cxlhk 8080:8080
> ```
>
> **Running from a local IDE against a remote workbench?** Also port-forward the Jupyter server:
> ```bash
> oc port-forward pod/summit-demo-0 8888:8888
> ```

## Setup

In [4]:
import json
import os
from pathlib import Path

from llama_stack_provider_trustyai_garak.utils import _ensure_xdg_vars

_ensure_xdg_vars()

assert "OPENAICOMPATIBLE_API_KEY" in os.environ, (
    "Set OPENAICOMPATIBLE_API_KEY before running. "
    "Export it in your shell or add: os.environ['OPENAICOMPATIBLE_API_KEY'] = '...'"
)

# Resolve repo root so this notebook works whether Jupyter was started from
# the repo root or from the notebooks/ subdirectory.
_cwd = Path.cwd()
repo_root = _cwd if (_cwd / "data" / "garak.yaml").exists() else _cwd.parent
config_path = repo_root / "data" / "garak.yaml"
assert config_path.exists(), f"Cannot find garak config: {config_path}"

print(f"Config:    {config_path}")

Config:    /Users/hjrnunes/workspace/redhat/hjrnunes/rh-summit-demos/data/garak.yaml


## Verify Garak data

Confirm that the intent typology produced by **garak-to-sdg.ipynb** is present before running the probe.

In [5]:
xdg_data = os.environ["XDG_DATA_HOME"]
typology_path = Path(xdg_data) / "garak" / "data" / "cas" / "trait_typology.json"

with open(typology_path) as f:
    typology = json.load(f)

print(f"Typology: {typology_path}")
print(f"Intents:  {list(typology.keys())}")

Typology: /Users/hjrnunes/workspace/redhat/hjrnunes/rh-summit-demos/tmp/garak/data/cas/trait_typology.json
Intents:  ['S001debtrepaymentnegotiation', 'S002executivecompensation', 'S003fraud', 'S004investmentadvice', 'S005moneylaundering', 'S006suspiciousactivityreporting']


## Run Garak

Invokes garak with `data/garak.yaml`, which specifies the target LLM, the probes (SPOIntent variants, TranslationIntent, TAPIntent), and the detector (Phi-4 as judge).

Garak reads the intent typology from `$XDG_DATA_HOME/garak/data/cas/trait_typology.json` automatically.
Report files are written to `$XDG_DATA_HOME/garak/garak_runs/`.

In [6]:
import garak.cli

garak.cli.main(["--config", str(config_path)])

garak LLM vulnerability scanner v0.14.1.pre1 ( https://github.com/NVIDIA/garak ) at 2026-02-25T16:36:50.076289
📜 logging to /Users/hjrnunes/workspace/redhat/hjrnunes/rh-summit-demos/tmp/garak/garak.log
📜 reporting to /Users/hjrnunes/workspace/redhat/hjrnunes/rh-summit-demos/tmp/garak/garak_runs/garak.cad5a346-c1a2-4b57-899a-8b740a19268b.report.jsonl
🌐 loading language services: zh,en->local.LocalHFTranslator[Helsinki-NLP/opus-mt-zh-en] en,zh->local.LocalHFTranslator[Helsinki-NLP/opus-mt-en-zh]
🎯 loading intent service
🎯 intents active: S001debtrepaymentnegotiation, S002executivecompensation, S003fraud, S004investmentadvice, S005moneylaundering, S006suspiciousactivityreporting


  0%|          | 0/2 [00:00<?, ?it/s]
OpenAICompatible ilyagusevgemma-2-9b-it-abliterated:   0%|          | 0/2 [00:00<?, ?it/s]
OpenAICompatible ilyagusevgemma-2-9b-it-abliterated:  50%|█████     | 1/2 [00:04<00:04,  4.52s/it]
OpenAICompatible ilyagusevgemma-2-9b-it-abliterated: 100%|██████████| 2/2 [00:08<00:00,  4.40s/it]
  0%|          | 0/2 [00:00<?, ?it/s]
OpenAICompatible ilyagusevgemma-2-9b-it-abliterated:   0%|          | 0/2 [00:00<?, ?it/s]
OpenAICompatible ilyagusevgemma-2-9b-it-abliterated:  50%|█████     | 1/2 [00:00<00:00,  1.39it/s]
OpenAICompatible ilyagusevgemma-2-9b-it-abliterated: 100%|██████████| 2/2 [00:01<00:00,  2.10it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
OpenAICompatible ilyagusevgemma-2-9b-it-abliterated:   0%|          | 0/2 [00:00<?, ?it/s]
OpenAICompatible ilyagusevgemma-2-9b-it-abliterated:  50%|█████     | 1/2 [00:02<00:02,  2.66s/it]
OpenAICompatible ilyagusevgemma-2-9b-it-abliterated: 100%|██████████| 2/2 [00:05<00:00,  2.49s/it]
  0%|          | 0/

📜 report closed :) /Users/hjrnunes/workspace/redhat/hjrnunes/rh-summit-demos/tmp/garak/garak_runs/garak.cad5a346-c1a2-4b57-899a-8b740a19268b.report.jsonl
📜 report html summary being written to /Users/hjrnunes/workspace/redhat/hjrnunes/rh-summit-demos/tmp/garak/garak_runs/garak.cad5a346-c1a2-4b57-899a-8b740a19268b.report.html
✔️  garak run complete in 241.09s


## Results

Locate the most recent report files produced by garak.

In [7]:
garak_runs = Path(os.environ["XDG_DATA_HOME"]) / "garak" / "garak_runs"
report_files = sorted(garak_runs.glob("*.report.jsonl"))
html_files   = sorted(garak_runs.glob("*.report.html"))

if report_files:
    print(f"JSONL report: {report_files[-1].resolve()}")
if html_files:
    print(f"HTML report:  {html_files[-1].resolve()}")

JSONL report: /Users/hjrnunes/workspace/redhat/hjrnunes/rh-summit-demos/tmp/garak/garak_runs/garak.cad5a346-c1a2-4b57-899a-8b740a19268b.report.jsonl
HTML report:  /Users/hjrnunes/workspace/redhat/hjrnunes/rh-summit-demos/tmp/garak/garak_runs/garak.cad5a346-c1a2-4b57-899a-8b740a19268b.report.html
